In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

#Import dt model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold

In [2]:
data = pd.read_csv('updated_data.csv')
data.head()

,tweet_text,retweet_count,favorite_count,followers_count,friends_count,verified,label,time_delay (min),neg_sent,pos_sent,neu_sent,comp_sent,difference,weight,ratio
0,"Reports of ""moving body"" amidst #Germanwings w...",38,15,337960,6384,1,1,1.2833,0.000,0.00,1.000,0.0000,0.0,0.0,1.000000e+00
1,BREAKING:148passengers were on board #GermanWi...,43,15,52815,293,0,1,1.5000,0.000,0.14,0.860,0.3818,1.0,1.0,1.000000e+10
2,BREAKING: #Germanwings crash victims include 7...,31,5,893549,2312,1,1,4.1333,0.278,0.00,0.722,-0.6124,-0.2,-0.2,6.666667e-01
3,BREAKING: 148 feared dead in crashed #Germanwi...,167,32,418641,1859,1,1,3.5167,0.517,0.00,0.483,-0.8176,0.0,0.0,1.000000e+00
4,Terrible news... Airbus A320 from Barcelona to...,26,1,11062,233,0,1,3.3667,0.171,0.00,0.829,-0.4767,0.0,0.0,1.000000e+00


In [3]:
text_data = data['tweet_text'].values
numerical_data = data[['retweet_count', 'followers_count', 'verified',
                           'time_delay (min)', 'neg_sent', 'pos_sent', 
                       'neu_sent', 'comp_sent', 'difference', 'ratio']].values
y = data['label']

In [36]:
# we added stratifying sampling
X_train_text, X_test_text, X_train_num, X_test_num, y_train, y_test = train_test_split(text_data, numerical_data, y, 
                                                                                       test_size=0.2, 
                                                                                       random_state=42, stratify=y)

In [37]:
# Get the shape of all the datasets
print(f'Train text: {len(X_train_text)}'), print(f'Train number: {len(X_train_num)}')
print(f'Test text: {len(X_test_text)}'), print(f'Test number:{len(X_test_num)}')
print(f'Y Training: {len(y_train)}'), print(f'Y Testing:{len(y_test)}')

Train text: 4641
Train number: 4641
Test text: 1161
Test number:1161
Y Training: 4641
Y Testing:1161


(None, None)

## Preprocess Data

In [6]:
import tensorflow_hub as hub
import tensorflow as tf

2024-07-26 13:50:18.330007: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 13:50:18.381081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 13:50:18.432784: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 13:50:18.445301: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 13:50:18.487844: I tensorflow/core/platform/cpu_feature_guar

In [9]:
import os
os.environ["CUDA_VISIBLE_DIVICES"]='-1'

In [10]:
elmo = hub.load("https://tfhub.dev/google/elmo/3")

In [11]:
def elmo_embeddings(text_list):
    embeddings = elmo.signatures['default'](tf.constant(text_list))['elmo']
    return embeddings

In [12]:
def process_in_batches(texts, batch_size=32):
    elmo_embed_np = []
    total_batches = (len(texts) + batch_size - 1) // batch_size  # Calculate total number of batches
    with tqdm(total=total_batches, desc='Processing Batches') as pbar:
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = elmo_embeddings(batch_texts)
            # Average the embeddings for each text
            batch_embeddings_np = [np.mean(embed.numpy(), axis=0) for embed in batch_embeddings]
            for embed in batch_embeddings_np:
                if embed.shape != (1024,):
                    print(f"Unexpected shape found: {embed.shape}")
            elmo_embed_np.extend(batch_embeddings_np)
            pbar.update(1)  # Update progress bar
    return np.array(elmo_embed_np)

In [38]:
train_elmo_embed_np = process_in_batches(X_train_text, batch_size=16)  # Adjust batch size as needed
test_elmo_embed_np = process_in_batches(X_test_text, batch_size=16)  # Adjust batch size as needed
test_elmo_embed_np.shape

Processing Batches: 100%|███████████████████████| 73/73 [03:35<00:00,  2.95s/it]


(1161, 1024)

## Numerical features

In [39]:
#using custom range
class CustomMinMaxScaler:
    def __init__(self, feature_range=(-2, 2)):
        self.feature_range = feature_range
        self.scaler = MinMaxScaler(feature_range=(0, 1))
    
    def fit(self, X):
        # Fit the scaler to the data
        self.scaler.fit(X)
        return self
    
    def transform(self, X):
        # Transform the data to [0, 1] range
        X_normalized = self.scaler.transform(X)
        # Scale to the desired range [-2, 2]
        a, b = self.feature_range
        X_scaled = a + (X_normalized * (b - a))
        return X_scaled
    
    def fit_transform(self, X):
        # Fit and transform the data
        return self.fit(X).transform(X)



In [40]:
num_scaler = CustomMinMaxScaler(feature_range=(-2, 2))
train_norm_num_features = num_scaler.fit_transform(X_train_num)
test_norm_num_features = num_scaler.fit_transform(X_test_num)
print(f'Normalized test shape: {test_norm_num_features.shape}')

Normalized test shape: (1161, 10)


## Concatenate Features

In [41]:
X_train = np.hstack((train_elmo_embed_np, train_norm_num_features))
X_test = np.hstack((test_elmo_embed_np, test_norm_num_features))
X_test.shape

(1161, 1034)

In [45]:
X_test.shape

(1161, 1034)

# Model Testing and CV

In [110]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)

param_grid = {
    'max_depth': [10, 20, 30, None]
}
grid_search = GridSearchCV(rf_model, param_grid, cv=10, verbose=0, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [10, 20, 30, None]}, scoring='accuracy')

In [111]:
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

Best parameters: {'max_depth': 10}


In [113]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

rf_op_model = RandomForestClassifier(max_depth=10, random_state=42) 

scoring = ['accuracy', 'recall',  'precision','f1_macro', 'f1_weighted' ]
scores = cross_validate(rf_op_model, X_train, y_train, scoring=scoring, cv=10)
rf_cv_scores = pd.DataFrame(scores)
display(rf_cv_scores)

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1_macro,test_f1_weighted
0,5.488741,0.010338,0.793548,0.588608,0.750000,0.755713,0.786519
1,5.490449,0.010716,0.829741,0.668790,0.795455,0.801506,0.825708
2,5.479976,0.010615,0.814655,0.636943,0.775194,0.782672,0.809624
3,5.502801,0.010426,0.795259,0.566879,0.767241,0.753488,0.786292
4,5.496793,0.010700,0.795259,0.620253,0.736842,0.762201,0.790482
5,5.491478,0.010570,0.801724,0.626582,0.750000,0.769279,0.796876
6,5.500451,0.010291,0.821121,0.626582,0.804878,0.788171,0.814819
7,5.559808,0.010341,0.812500,0.607595,0.793388,0.777060,0.805412
8,5.666005,0.010435,0.808190,0.658228,0.748201,0.779645,0.804942
9,5.506227,0.010679,0.816810,0.670886,0.762590,0.789549,0.813709


In [116]:
avg_f1_macro_cv_score = rf_cv_scores['test_f1_macro'].mean()
avg_f1_weighted_cv_score = rf_cv_scores['test_f1_weighted'].mean()
avg_precision_cv_score = rf_cv_scores['test_precision'].mean()
avg_recall_cv_score = rf_cv_scores['test_recall'].mean()
avg_acc_cv_score = rf_cv_scores['test_accuracy'].mean()

acc_perc = round((avg_acc_cv_score * 100), 2)
recall_perc = round((avg_recall_cv_score * 100), 2)
precision_perc = round((avg_precision_cv_score * 100), 2)
weighted_perc = round((avg_f1_weighted_cv_score * 100), 2)
macro_perc = round((avg_f1_macro_cv_score * 100), 2)

print(f'Precision: {avg_precision_cv_score}', f'| Prec %: {precision_perc}')
print(f'Recall: {avg_recall_cv_score}', f' | Recall %: {recall_perc}')
print(f'Accuracy: {avg_acc_cv_score}', f' | Accuracy %: {acc_perc}')
print(f'Macro F1: {avg_f1_macro_cv_score}', f' | Macro F1 %: {macro_perc}')
print(f'Weighted F1: {avg_f1_weighted_cv_score}', f' | Weighted F1 %: {weighted_perc}')

Precision: 0.7683789673916689 | Prec %: 76.84
Recall: 0.6271345642183344  | Recall %: 62.71
Accuracy: 0.8088807007786428  | Accuracy %: 80.89
Macro F1: 0.7759285609074636  | Macro F1 %: 77.59
Weighted F1: 0.8034382288456751  | Weighted F1 %: 80.34


In [118]:
rf_op_model.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [120]:
y_test_predict = rf_op_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)

test_acc_perc = round((test_accuracy * 100), 2)
test_recall_perc = round((test_recall * 100), 2)
test_precision_perc = round((test_precision * 100), 2)

print(f'Test Precision: {test_precision}', f" | Test Set Accuracy %: {test_acc_perc}")
print(f'Training Set Recall: {test_recall}', f' | Test Set Recall %: {test_recall_perc}')
print(f"Training Set Accuracy: {test_accuracy}", f' | Test Set Precision %: {test_precision_perc}')

from sklearn.metrics import f1_score
macrof1_score = f1_score(y_test, y_test_predict, average='macro')
macro_perc = round((macrof1_score * 100), 2)
print(f'Macro F1: {macrof1_score}',f' | Test Macro F1 %: {macro_perc}')

wf1_score = f1_score(y_test, y_test_predict, average='weighted')
weighted_perc = round((wf1_score * 100), 2)
print(f'Weighted F1: {wf1_score}',f' | Test Weighted F1 %: {weighted_perc}')

Test Precision: 0.7645161290322581  | Test Set Accuracy %: 80.1
Training Set Recall: 0.6  | Test Set Recall %: 60.0
Training Set Accuracy: 0.8010335917312662  | Test Set Precision %: 76.45
Macro F1: 0.75580931796561  | Test Macro F1 %: 75.58
Weighted F1: 0.7867262498896539  | Test Weighted F1 %: 78.67


# Model

In [20]:
def rf_cv(max_depth, min_samples_split, min_samples_leaf, criterion):
    #defining the mapping of the criterion
    criterion_mapping = {0: 'gini', 1:'entropy'}
    
    max_depth = int(round(max_depth))
    min_samples_split = int(round(min_samples_split))
    min_samples_leaf = int(round(min_samples_leaf))
    criterion = criterion_mapping[int(round(criterion))]

    # define the model
    model = RandomForestClassifier(
        max_depth = max_depth,
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf,
        criterion = criterion,
        bootstrap = True,
        random_state = 123)
    
    kfold = StratifiedKFold(n_splits = 5, shuffle=True, random_state=123)
    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=kfold)

    score = np.nan_to_num(score)
    score = score.mean()
    return score

In [21]:
rf_params = {
    'max_depth': (5, 50),
    'min_samples_split': (2, 100), 
    'min_samples_leaf': (1, 50),
    'criterion':(0, 1)
}

In [22]:
rf_bo_optimizer = BayesianOptimization(rf_cv, rf_params,random_state=111)
rf_bo_optimizer.maximize(n_iter=100)

|   iter    |  target   | criterion | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------------------
| 1         | 0.7919    | 0.6122    | 12.61     | 22.37     | 77.39     |
| 2         | 0.8003    | 0.2953    | 11.71     | 2.101     | 43.18     |
| 3         | 0.7867    | 0.2387    | 20.19     | 49.54     | 25.3      |
| 4         | 0.7901    | 0.08119   | 35.13     | 31.44     | 28.88     |
| 5         | 0.7966    | 0.4662    | 10.33     | 4.624     | 90.28     |
| 6         | 0.7891    | 0.8909    | 5.396     | 1.0       | 28.25     |
| 7         | 0.7908    | 0.6486    | 42.74     | 19.92     | 96.78     |
| 8         | 0.7891    | 0.7168    | 5.962     | 47.38     | 23.47     |
| 9         | 0.7923    | 0.2423    | 10.21     | 5.649     | 89.25     |
| 10        | 0.7964    | 0.9233    | 11.4      | 3.769     | 42.71     |
| 11        | 0.8033    | 0.385     | 11.29     | 1.555     | 41.32     |
| 12        | 0.8028    | 0.268     | 

In [23]:
# # Print the best result
print(rf_bo_optimizer.max)

{'target': 0.8103861215248134, 'params': {'criterion': 0.15208264633275181, 'max_depth': 22.911937949261837, 'min_samples_leaf': 2.656076580613511, 'min_samples_split': 8.324635540417855}}


In [24]:
# Extract the results
results = rf_bo_optimizer.res
# Create a DataFrame from the results
results_df = pd.DataFrame(results)
# Optionally, save the DataFrame to a CSV file
results_df.to_csv('rf_bayesian_optimization_results.csv', index=False)

In [25]:
import json
# Extract the best result
best_result = rf_bo_optimizer.max

# Save the best result to a JSON file
with open('rf_best_result.json', 'w') as f:
    json.dump(best_result, f)

# Model Performance with Optimal Parameters

In [70]:
print(int(round(max_depth)))

23
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [99]:
criterion = 0.15208264633275181
max_depth = 22.911937949261837
min_samples_leaf = 2.656076580613511
min_samples_split = 8.324635540417855

In [100]:
def rf_model(max_depth, min_samples_split, min_samples_leaf, criterion):
    #defining the mapping of the criterion
    criterion_mapping = {0: 'gini', 1:'entropy'}
    
    max_depth = int(round(max_depth))
    min_samples_split = int(round(min_samples_split))
    min_samples_leaf = int(round(min_samples_leaf))
    criterion = criterion_mapping[int(round(criterion))]

    # define the model
    model = RandomForestClassifier(
        max_depth = max_depth,
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf,
        criterion = criterion,
        bootstrap = True,
        random_state = 123)
    return model

In [101]:
model = rf_model(max_depth, min_samples_split, min_samples_leaf, criterion)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=23, min_samples_leaf=3, min_samples_split=8,
                       random_state=123)

### Test

In [102]:
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Set Accuracy: {train_accuracy}")

train_recall = recall_score(y_train, y_train_pred)
print(f'Training Set Recall: {train_recall}')

train_precision = precision_score(y_train, y_train_pred)
print(f'Training Set Precision: {train_precision}')

Training Set Accuracy: 0.9931049342814049
Training Set Recall: 0.979708306911858
Training Set Precision: 1.0


In [103]:
train_acc_perc = round((train_accuracy * 100), 2)
train_recall_perc = round((train_recall * 100), 2)
train_precision_perc = round((train_precision * 100), 2)

print(f"Training Set Accuracy %: {train_acc_perc}")
print(f'Training Set Recall %: {train_recall_perc}')
print(f'Training Set Precision %: {train_precision_perc}')

Training Set Accuracy %: 99.31
Training Set Recall %: 97.97
Training Set Precision %: 100.0


In [104]:
from sklearn.metrics import f1_score
train_macrof1_score = f1_score(y_train, y_train_pred, average='macro')
train_macro_perc = round((train_macrof1_score * 100), 2)
print(train_macrof1_score)

print(f'Macro F1 %: {train_macro_perc}')

train_wf1_score = f1_score(y_train, y_train_pred, average='weighted')
train_weighted_perc = round((train_wf1_score * 100), 2)
print(train_wf1_score)

print(f'Weighted F1 %: {train_weighted_perc}')

0.9922776774794713
Macro F1 %: 99.23
0.9930875068833391
Weighted F1 %: 99.31


### Test

In [105]:
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Training Set Accuracy: {test_accuracy}")

test_recall = recall_score(y_test, y_test_pred)
print(f'Training Set Recall: {test_recall}')

test_precision = precision_score(y_test, y_test_pred)
print(f'Training Set Precision: {test_precision}')

Training Set Accuracy: 0.8010335917312662
Training Set Recall: 0.6
Training Set Precision: 0.7645161290322581


In [106]:
test_acc_perc = round((test_accuracy * 100), 2)
test_recall_perc = round((test_recall * 100), 2)
test_precision_perc = round((test_precision * 100), 2)

print(f"Test Set Accuracy %: {test_acc_perc}")
print(f'Test Set Recall %: {test_recall_perc}')
print(f'Test Set Precision %: {test_precision_perc}')

Test Set Accuracy %: 80.1
Test Set Recall %: 60.0
Test Set Precision %: 76.45


In [107]:
from sklearn.metrics import f1_score
test_macrof1_score = f1_score(y_test, y_test_pred, average='macro')
test_macro_perc = round((test_macrof1_score * 100), 2)
print(test_macrof1_score)

print(f'Macro F1 %: {test_macro_perc}')

test_wf1_score = f1_score(y_test, y_test_pred, average='weighted')
test_weighted_perc = round((test_wf1_score * 100), 2)
print(test_wf1_score)

print(f'weighted F1 %: {test_weighted_perc}')

0.764741641337386
Macro F1 %: 76.47
0.79426864483767
weighted F1 %: 79.43


# Baseline model performance

In [94]:
base_model = RandomForestClassifier(
        min_samples_split = 2,
        min_samples_leaf = 1,
        criterion = 'gini',
        max_depth = 23,
        bootstrap = True,
        random_state = 123)
base_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=23, random_state=123)

In [89]:
base_model = RandomForestClassifier()
base_model.fit(X_train, y_train)

RandomForestClassifier()

### Training

In [95]:
base_train_pred = base_model.predict(X_train)
base_train_accuracy = accuracy_score(y_train, base_train_pred)
print(f"Training Set Accuracy: {base_train_accuracy}")

base_train_recall = recall_score(y_train, base_train_pred)
print(f'Training Set Recall: {base_train_recall}')

base_train_precision = precision_score(y_train, base_train_pred)
print(f'Training Set Precision: {base_train_precision}')

Training Set Accuracy: 0.9995690583925878
Training Set Recall: 0.9987317691819911
Training Set Precision: 1.0


In [96]:
from sklearn.metrics import f1_score
base_train_acc_perc = round((base_train_accuracy * 100), 2)
base_train_recall_perc = round((base_train_recall * 100), 2)
base_train_precision_perc = round((base_train_precision * 100), 2)

print(f"Training Set Accuracy %: {base_train_acc_perc}")
print(f'Training Set Recall %: {base_train_recall_perc}')
print(f'Training Set Precision %: {base_train_precision_perc}')

base_train_macrof1_score = f1_score(y_train, base_train_pred, average='macro')
base_train_macro_perc = round((base_train_macrof1_score * 100), 2)
print(base_train_macrof1_score)

print(f'Macro F1 %: {base_train_macro_perc}')

base_train_wf1_score = f1_score(y_train, base_train_pred, average='weighted')
base_train_weighted_perc = round((base_train_wf1_score * 100), 2)
print(base_train_wf1_score)

print(f'Weighted F1 %: {base_train_weighted_perc}')

Training Set Accuracy %: 99.96
Training Set Recall %: 99.87
Training Set Precision %: 100.0
0.9995196089797203
Macro F1 %: 99.95
0.9995689919729601
Weighted F1 %: 99.96


### Testing

In [97]:
base_y_test_pred = base_model.predict(X_test)
base_test_accuracy = accuracy_score(y_test, base_y_test_pred)
print(f"Training Set Accuracy: {base_test_accuracy}")

base_test_recall = recall_score(y_test, base_y_test_pred)
print(f'Training Set Recall: {base_test_recall}')

base_test_precision = precision_score(y_test, base_y_test_pred)
print(f'Training Set Precision: {base_test_precision}')

Training Set Accuracy: 0.8096468561584841
Training Set Recall: 0.6075949367088608
Training Set Precision: 0.7843137254901961


In [98]:
base_test_acc_perc = round((base_test_accuracy * 100), 2)
base_test_recall_perc = round((base_test_recall * 100), 2)
base_test_precision_perc = round((base_test_precision * 100), 2)

print(f"Test Set Accuracy %: {base_test_acc_perc}")
print(f'Test Set Recall %: {base_test_recall_perc}')
print(f'Test Set Precision %: {base_test_precision_perc}')

base_test_macrof1_score = f1_score(y_test, base_y_test_pred, average='macro')
base_test_macro_perc = round((base_test_macrof1_score * 100), 2)
print(base_test_macro_perc)

print(f'Macro F1 %: {base_test_macro_perc}')

base_test_wf1_score = f1_score(y_test, base_y_test_pred, average='weighted')
base_test_weighted_perc = round((base_test_wf1_score * 100), 2)
print(base_test_weighted_perc)

print(f'Weighted F1 %: {base_test_weighted_perc}')

Test Set Accuracy %: 80.96
Test Set Recall %: 60.76
Test Set Precision %: 78.43
77.42
Macro F1 %: 77.42
80.28
Weighted F1 %: 80.28


# Cross Validation

In [108]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['accuracy', 'recall',  'precision','f1_macro', 'f1_weighted' ]
scores = cross_validate(base_model, X_train, y_train, scoring=scoring)
base_cv_scores = pd.DataFrame(scores)
display(base_cv_scores)

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1_macro,test_f1_weighted
0,6.689531,0.016381,0.806243,0.629747,0.759542,0.773978,0.801279
1,6.564184,0.016166,0.815733,0.631746,0.783465,0.783303,0.810222
2,6.595619,0.016007,0.784483,0.615873,0.710623,0.751068,0.780355
3,7.134429,0.019752,0.812500,0.612698,0.787755,0.777513,0.805845
4,7.237105,0.016063,0.814655,0.661392,0.762774,0.786307,0.811133


In [109]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['accuracy', 'recall',  'precision','f1_macro', 'f1_weighted' ]
scores = cross_validate(model, X_train, y_train, scoring=scoring)
op_cv_scores = pd.DataFrame(scores)
display(op_cv_scores)

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1_macro,test_f1_weighted
0,5.800952,0.015815,0.799785,0.598101,0.762097,0.763236,0.792976
1,5.771056,0.015637,0.814655,0.631746,0.780392,0.782249,0.809224
2,5.742748,0.015260,0.788793,0.606349,0.726236,0.753768,0.783589
3,5.763300,0.015325,0.811422,0.593651,0.799145,0.773672,0.803354
4,5.699537,0.015187,0.804957,0.636076,0.752809,0.773677,0.800514
